In [15]:
import pandas as pd
import itertools

import os
import shutil

from dotenv import load_dotenv
load_dotenv()

True

### Helper Functions

In [78]:
def generate_decklists(cards_df):
    """
    Creates all possible Jumpstart decks (two half decks put together), then generates output .dck files for use in Forge

    Args:
        cards_df (DataFrame): Dataframe containing cards categorized into decks

    Returns:
        String: Path to output folder
    """
    deck_names = cards_df['deck'].unique()
    deck_combinations = itertools.combinations(deck_names, 2)
    output_path = os.path.join('output', 'jumpstart')

    for deck1_name, deck2_name in deck_combinations:
        deck1 = cards_df[cards_df['deck'] == deck1_name]
        deck2 = cards_df[cards_df['deck'] == deck2_name]
        deck = pd.concat([deck1, deck2])

        generate_deck_file(deck, deck1_name+" "+deck2_name, output_path)

    return output_path


def generate_deck_file(deck, name='Sample Deck', output_path='output/jumpstart'):
    """
    Saves deck as a .dck file for Forge

    Args:
        deck (DataFrame): Dataframe containing cards in a deck
        name (String): Deck name
        output_path (String): Path to output folder
    """
    os.makedirs(output_path, exist_ok=True)

    with open(os.path.join(output_path, f"{name}.dck"), 'w') as f:
        f.write('[metadata]\n')
        f.write(f'Name={name}\n')
        f.write('[Avatar]\n\n')
        f.write('[Main]\n')
        for _, row in deck.iterrows():
            set_code = row['set_code'] if row['set_code'] else ''
            f.write(f"{row['quantity']} {row['name']}|{set_code}|1\n")

        f.write('[Sideboard]\n\n')
        f.write('[Planes]\n\n')
        f.write('[Schemes]\n\n')
        f.write('[Conspiracy]\n\n')
        f.write('[Dungeon]')


def add_lands(cards_df):
    """
    Adds lands to decks that do not have lands. Tedious to do in Archidekt, so it's automated!

    Args:
        cards_df (DataFrame): Dataframe containing cards categorized into decks

    Returns:
        DataFrame: Dataframe containing cards categorized into decks, with added lands
    """
    deck_names = cards_df['deck'].unique()
    lands = {
        'W': [
            {
                'quantity': '7',
                'name':     'Plains',
                'set_code': 'JMP',
                'tag':      'Land',
            },
            {
                'quantity': '1',
                'name':     'Thriving Heath',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'U': [
            {
                'quantity': '7',
                'name':     'Island',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Isle',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'B': [
            {
                'quantity': '7',
                'name':     'Swamp',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Moor',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'R': [
            {
                'quantity': '7',
                'name':     'Mountain',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Bluff',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'G': [
            {
                'quantity': '7',
                'name':     'Forest',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Grove',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
    }

    for deck in deck_names:
        deck_df = cards_df[cards_df['deck'] == deck]
        colour = deck_df['colour'].iloc[0]

        if deck_df[deck_df['name'].isin(['Plains', 'Island', 'Swamp', 'Mountain', 'Forest'])].empty:
            lands_df = pd.DataFrame.from_dict(lands[colour])
            lands_df['deck'] = deck

            cards_df = pd.concat([cards_df, lands_df])


    return cards_df


def parse_card(card):
    """
    Parses an individual card line from input CSV

    Args:
        card (String): Individual card line from CSV

    Returns:
        Dictionary: Card line parsed into dictionary
    """
    split_card = card.split()

    card_dict = {
        'quantity': '',
        'name':     '',
        'colour':   '',
        'set_code': '',
        'deck':     '',
        'tag':      '',
    }

    card_dict['quantity'] = split_card[0].split('x')[0]

    step = 0
    for index, word in enumerate(split_card[1:]):
        if step == 0 and '(' not in word:
            card_dict['name'] = card_dict['name'] + ' ' + word
        elif step == 0:
            step = 1

        if step == 1:
            card_dict['set_code'] = card_dict['set_code'] + ' ' + word.strip('()').upper()
            if ')' in word:
                step = 2
                continue

        if step == 2:
            deck_name = word.strip('[]')
            card_dict['deck'] = card_dict['deck'] + ' ' + deck_name
            if ' - ' in card_dict['deck']:
                card_dict['colour'] = card_dict['deck'].split(' - ')[0].strip()
                card_dict['deck'] = card_dict['deck'].split(' - ')[1].strip()
            if ']' in word:
                step = 3
            continue

        if step == 3:
            card_dict['tag'] = word.split(',')[0].strip('^')

    for key in card_dict:
        card_dict[key] = card_dict[key].strip().replace('/', '')

    return card_dict


def parse_decks(cards):
    """
    Parses an input CSV

    Args:
        cards (List): List of card lines from CSV

    Returns:
        DataFrame: All cards processed
    """
    card_list = [parse_card(card) for card in cards]
    cards_df = pd.DataFrame.from_dict(card_list)

    return cards_df

### Process Decks

In [81]:
with open('input/jumpstart.txt', 'r') as file:
    cards = file.readlines()

cards_df = parse_decks(cards)
cards_df = add_lands(cards_df)

assert len(cards_df['deck'].unique()) == cards_df['quantity'].astype(int).sum() // 20, "Deck count does not match expected value (cards/20)"

deck_path = generate_decklists(cards_df)
forge_path = os.getenv("FORGE_DECKS_PATH")

# Clear the folder at FORGE_PATH
if forge_path and os.path.exists(forge_path):
    for filename in os.listdir(forge_path):
        file_path = os.path.join(forge_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

# Copy contents of deck_path to FORGE_PATH
if forge_path and os.path.exists(deck_path):
    for filename in os.listdir(deck_path):
        src_file = os.path.join(deck_path, filename)
        dst_file = os.path.join(forge_path, filename)
        shutil.copy2(src_file, dst_file)

### Evaluate Decks